# Обработка из общего файла в отдельные таблицы

In [ ]:
import pandas as pd
import os

# Папка, где находится ваш CSV-файл
data_folder = '.'  # Текущая папка
file_name = 'output_data.csv'  # Общая таблица, полученная кодом моего коллеги, содержащая отдельную информацию по каждому абитуриенту
file_path = os.path.join(data_folder, file_name)

# Загрузка данных из CSV с low_memory=False
df = pd.read_csv(file_path, low_memory=False)

# Удаление строк, где в столбце "Фамилия, имя, отчество" стоит NaN
df = df.dropna(subset=['Фамилия, имя, отчество'])

# Убедитесь, что столбцы, по которым группируем, не содержат пропуски
df = df.dropna(subset=['Год', 'Факультет', 'Специальность', 'Положение студента'])

# Создание папки для результатов, если необходимо
output_folder = 'split_tables'
os.makedirs(output_folder, exist_ok=True)

# Группировка по годам, факультетам и специальностям
grouped = df.groupby(['Год', 'Факультет', 'Специальность', 'Положение студента'])

# Замена NaN в непустых столбцах
for (year, faculty, specialty, situation), group in grouped:
    # Удаление столбцов, в которых все значения NaN
    group = group.dropna(axis=1, how='all')

    # Убедимся, что изменение применяется к оригиналу данных
    for col in group.columns:
        if group[col].isna().sum() < len(group):
            group.loc[:, col] = group[col].fillna(float('nan'))
    
    # Создание уникального имени файла, чтобы избежать ошибок файловой системы
    file_name = f"{year}_{faculty}_{specialty}_{situation}.csv"

    # Сохранение в CSV
    output_path = os.path.join(output_folder, file_name)
    group.to_csv(output_path, index=False)

print("Разделение данных завершено. Файлы сохранены в папке 'split_tables'.")

# Обработка из отдельных таблиц в общую

## Создание общей таблицы

In [ ]:
import pandas as pd
import os

# Папка, где находятся исходные данные
data_folder = 'split_tables'  # Папка с вашими CSV-файлами
input_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Имена столбцов для общей таблицы
summary_columns = [
    'год', 'факультет', 'специальность', 'кол-во бви', 'кол-во целевиков', 
    'кол-во квотников', 'кол-во участников конкурса', 'проходной балл', 'средний балл', 
    'гуманитарность факультета', 'кол-во букв в аббревиатуре', 'кол-во бюджетных мест', 
    'гендер', 'спец или бак', 'процент немосквичей', 'средние доп баллы', 'кол-во бронзы ГТО'
]

# Создаем пустой DataFrame с заданной структурой
summary_df = pd.DataFrame(columns=summary_columns)

# Заполнение общей таблицы на основе данных из файлов
for input_file in input_files:
    file_path = os.path.join(data_folder, input_file)
    df = pd.read_csv(file_path)  # Загрузка данных из CSV
    
    # Извлечение информации из имени файла
    file_info = input_file.replace('.csv', '').split('_')
    year, faculty, specialty = int(file_info[0]), file_info[1], file_info[2]
    
    # Создание строки для добавления в общую таблицу
    new_row = {
        'год': year,
        'факультет': faculty,
        'специальность': specialty,
        'кол-во бви': 0,
        'кол-во целевиков': 0,
        'кол-во квотников': 0,
        'кол-во участников конкурса': 0,
        'проходной балл': None,
        'средний балл': None,
        'гуманитарность факультета': None,
        'кол-во букв в аббревиатуре': len(specialty),  # Можно посчитать
        'кол-во бюджетных мест': None,
        'гендер': None,
        'спец или бак': None,
        'процент немосквичей': None,
        'средние доп баллы': None,
        'кол-во бронзы ГТО': 0
    }
    
    # Добавление новой строки в общую таблицу с использованием concat
    summary_df = pd.concat([summary_df, pd.DataFrame([new_row])], ignore_index=True)

# Удаление дубликатов, если они есть (группировка по году, факультету и специальности)
summary_df = summary_df.drop_duplicates(subset=['год', 'факультет', 'специальность'])

# Сохранение общей таблицы в CSV
output_path = os.path.join('.', 'общая_таблица.csv')
summary_df.to_csv(output_path, index=False)

print("Общая таблица создана и сохранена.")

## Заполнение данных из отдельных файлов

In [ ]:
import pandas as pd
import os

# Папка, где находится общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Папка с таблицами, которые нужно обработать
input_folder = 'split_tables'  # Папка с отдельными таблицами
input_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Функция для извлечения информации из имени файла
def extract_info_from_filename(filename):
    # Предполагаем, что имя файла имеет формат "год_факультет_специальность_прочее.csv"
    parts = filename.split('_')
    # Извлекаем год, факультет и специальность
    year = int(parts[0])  # Первый элемент - год
    faculty = parts[1]  # Второй элемент - факультет
    specialty = parts[2]  # Третий элемент - специальность
    return year, faculty, specialty

# Функция для обновления соответствующих значений в общей таблице
def update_summary_table(summary_df, year, faculty, specialty, updates):
    condition = (
        (summary_df['год'] == year) &
        (summary_df['факультет'] == faculty) &
        (summary_df['специальность'] == specialty)
    )
    
    # Обновление значений в зависимости от предоставленных данных
    for key, value in updates.items():
        if key in summary_df.columns:
            summary_df.loc[condition, key] = round(value, 4)



# Обработка каждой таблицы
for input_file in input_files:
    file_path = os.path.join(input_folder, input_file)
    df = pd.read_csv(file_path)  # Загрузка отдельной таблицы

    # Извлечение информации из имени файла
    year, faculty, specialty = extract_info_from_filename(input_file)

    # Определение количества строк
    row_count = len(df)

    updates = {}

    if "без вступительных испытаний" in input_file:
        updates['кол-во бви'] = row_count
    
    elif "по целевой квоте" in input_file:
        updates['кол-во целевиков'] = row_count
    
    elif "по особой квоте (социальные льготы)" in input_file:
        updates['кол-во квотников'] = row_count
    
    elif "по общему конкурсу" in input_file:
        updates['кол-во участников конкурса'] = row_count
        updates['средний балл'] = df['Сумма баллов'].mean()  # Средний балл
        updates['кол-во бюджетных мест'] = len(df[df['Статус'] == 'Зачислен'])  # Бюджетные места
        if len(df[df['Статус'] == 'Зачислен']) > 0:
            updates['проходной балл'] = df[df['Статус'] == 'Зачислен']['Сумма баллов'].iloc[-1] 
        updates['гендер'] = df['Пол'].mean()  # Среднее значение пола
        updates['процент немосквичей'] = df['Право на общежитие'].apply(lambda x: 1 if x == 'Да' else 0).mean()  # Процент немосквичей
        updates['средние доп баллы'] = df[['Аттестат с отличием ', 'Оценка за сочинение ', 'ГТО (спортивные достижения)']].sum(axis=1).mean()  # Средние доп. баллы

    # Обновление общей таблицы с нужными значениями
    update_summary_table(summary_df, year, faculty, specialty, updates)

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)  # Путь для сохранения
summary_df.to_csv(summary_file_path, index=False)

print("Обработка таблиц завершена. Общая таблица обновлена.")

## Заполнение гуманитарности

In [ ]:
import pandas as pd
import os

# Список гуманитарных факультетов
humanities = {
    'Философский факультет',
    'Факультет политологии',
    'Факультет иностранных языков и регионоведения',
    'Юридический факультет',
    'Филологический факультет',
    'Факультет глобальных процессов',
    'Исторический факультет',
    'Высшая школа телевидения',
    'Высшая школа перевода',
    'Высшая школа культурной политики и управления в гуманитарной сфере',
    'Факультет государственного управления',
    'Социологический факультет',
    'Институт стран Азии и Африки',
    'Факультет журналистики',
    'Экономический факультет',
    'Факультет искусств',
    'Высшая школа современных социальных наук',
    'Факультет мировой политики',
}

# Папка, где находится общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Добавление столбца "гуманитарность"
summary_df['гуманитарность факультета'] = summary_df['факультет'].apply(lambda x: 1 if x in humanities else 0)

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Общая таблица обновлена. Гуманитарность факультетов определена.")

## Заполнение инструмента

In [ ]:
import pandas as pd
import os

# Папка, где находится файл Excel
data_folder = '.'  # Текущая папка
instrument_file_name = 'инструмент.xlsx'  # Имя файла Excel
instrument_file_path = os.path.join(data_folder, instrument_file_name)

# Загрузка данных из Excel
instrument_df = pd.read_excel(instrument_file_path)

# Исправление названий факультетов
correction_map = {
    'Юрфак': 'Юридический факультет',
    'Эконом': 'Экономический факультет',
    'физфак': 'Физический факультет',
    'биологи': 'Биологический факультет',
    'химики': 'Химический факультет',
    'фмп(мировой политики)': 'Факультет мировой политики',
    'журфак': 'Факультет журналистики',
    'вшга(аудит)': 'Высшая школа государственного аудита',
    'геологи': 'Геологический факультет',
    'биотехнологии': 'Биотехнологический факультет',
    'географы': 'Географический факультет',
    'историки': 'Исторический факультет',
    'вмк': 'Факультет вычислительной математики и кибернетики',
    'исаа(африка и азия)': 'Институт стран Азии и Африки',
    'мехмат': 'Механико–математический факультет'
}

# Применяем исправление к столбцу факультета
instrument_df['факультет'] = instrument_df['факультет'].str.strip().map(correction_map)

# Загрузка общей таблицы
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df = pd.read_csv(summary_file_path)

# Обновление столбца "кол-во бронзы ГТО"
bronze_data = instrument_df[['год', 'факультет', 'число резов 1курсников на бронзу гто бег + лыжники']]
bronze_data.rename(columns={'число резов 1курсников на бронзу гто бег + лыжники': 'кол-во бронзы ГТО'}, inplace=True)

# Обновляем общую таблицу с использованием корректированных названий факультетов
for _, row in bronze_data.iterrows():
    year = row['год']
    faculty = row['факультет']
    bronze_count = row['кол-во бронзы ГТО']

    # Обновление столбца в общей таблице
    condition = (summary_df['факультет'] == faculty) & (summary_df['год'] == year)
    summary_df.loc[condition, 'кол-во бронзы ГТО'] = bronze_count

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Общая таблица обновлена. Данные о количестве бронзы ГТО добавлены.")

## Заполнение проходного балла

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Загрузка данных из Excel-файла "Проходные баллы 2016-2019"
passing_scores_file_name = 'Проходные баллы 2016-2019.xlsx'
passing_scores_file_path = os.path.join(data_folder, passing_scores_file_name)

# Чтение данных из Excel, игнорируя первые строки с нежелательными заголовками
passing_scores_df = pd.read_excel(passing_scores_file_path, header=1)  # Устанавливаем правильный заголовок

# Переименование столбцов
passing_scores_df.columns = ['Факультет', 'Уровень образования', 'Форма обучения', 'Другая специальность', 'Специальность', '2016_1', '2016_2', '2017_1', '2017_2', '2018_1', '2018_2', '2019_1', '2019_2']

# Преобразование названий факультетов к согласованному виду
passing_scores_df['Факультет'] = passing_scores_df['Факультет'].str.capitalize()
passing_scores_df['Специальность'] = passing_scores_df['Специальность'].str.capitalize()

# Преобразование уровня образования
level_map = {'СПЕЦИАЛИСТЫ': 'СПЕЦИАЛИСТЫ', 'БАКАЛАВРЫ': 'БАКАЛАВРЫ'}
passing_scores_df['Уровень образования'] = passing_scores_df['Уровень образования'].map(level_map)

# Функция для обновления общей таблицы для всех годов
def update_summary_table_for_years(summary_df, passing_scores_df, year, col_second_wave):
    # Извлекаем данные о проходных баллах второй волны
    data_for_year = passing_scores_df[['Факультет', 'Специальность', col_second_wave]]
    
    for _, row in data_for_year.iterrows():
        faculty = row['Факультет']
        specialty = row['Специальность']
        passing_score = row[col_second_wave]
        
        if pd.isna(passing_score):
            continue  # Пропускаем строки, где нет данных о проходном балле
        
        # Условие для поиска нужной строки в общей таблице
        condition = (
            (summary_df['год'] == year) &
            (summary_df['факультет'] == faculty) &
            (summary_df['специальность'] == specialty)
        )
        
        # Обновление проходного балла второй волны
        if summary_df.loc[condition, 'проходной балл'].isna().any():
            summary_df.loc[condition, 'проходной балл'] = passing_score

# Обновление общей таблицы для каждого года
years = {
    2016: '2016_2',
    2017: '2017_2',
    2018: '2018_2',
    2019: '2019_2'
}

# Проходной балл второй волны для каждого года
for year, col_second_wave in years.items():
    update_summary_table_for_years(summary_df, passing_scores_df, year, col_second_wave)

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Общая таблица обновлена для всех годов.")

## Заполнение формы обучения

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Загрузка данных из Excel-файла "Проходные баллы 2016-2019"
passing_scores_file_name = 'Проходные баллы 2016-2019.xlsx'
passing_scores_file_path = os.path.join(data_folder, passing_scores_file_name)

# Чтение данных из Excel, игнорируя первые строки с нежелательными заголовками
passing_scores_df = pd.read_excel(passing_scores_file_path, header=1)  # Устанавливаем правильный заголовок

# Переименование столбцов
passing_scores_df.columns = ['Факультет', 'Уровень образования', 'Форма обучения', 'Другая специальность', 'Специальность', '2016_1', '2016_2', '2017_1', '2017_2', '2018_1', '2018_2', '2019_1', '2019_2']

# Преобразование названий факультетов к согласованному виду
passing_scores_df['Факультет'] = passing_scores_df['Факультет'].str.capitalize()
passing_scores_df['Специальность'] = passing_scores_df['Специальность'].str.capitalize()

# Преобразование уровня образования
level_map = {'СПЕЦИАЛИСТЫ': 1, 'БАКАЛАВРЫ': 0}
passing_scores_df['Уровень образования'] = passing_scores_df['Уровень образования'].map(level_map)

# Обновление столбца "спец или бак" в общей таблице
for _, row in passing_scores_df.iterrows():
    faculty = row['Факультет']
    specialty = row['Специальность']
    level_of_education = row['Уровень образования']

    # Условие для поиска нужной строки в общей таблице
    condition = (
        (summary_df['факультет'] == faculty) &
        (summary_df['специальность'] == specialty)
    )
    
    # Обновление значения столбца "спец или бак" в общей таблице
    summary_df.loc[condition, 'спец или бак'] = level_of_education

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Столбец 'спец или бак' в общей таблице обновлен без предупреждений.")

## Удаление пустых строк

In [ ]:
df = pd.read_csv('общая_таблица.csv')
df = df.dropna(how='any').reset_index().drop('index', axis=1)  # Удалить полностью пустые строки
df.to_csv('общая_таблица.csv')
df

## Переименование столбцов

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Словарь с переименованными столбцами
column_rename_map = {
    'год': 'year',
    'факультет': 'faculty',
    'специальность': 'specialty',
    'кол-во бви': 'num_bvi',
    'кол-во целевиков': 'num_target_students',
    'кол-во квотников': 'num_quota_students',
    'кол-во участников конкурса': 'num_student',
    'проходной балл': 'passing_score',
    'средний балл': 'average_score',
    'гуманитарность факультета': 'faculty_humanities',
    'кол-во букв в аббревиатуре': 'abbreviation_length',
    'кол-во бюджетных мест': 'num_budget_places',
    'гендер': 'gender',
    'спец или бак': 'specialist_or_bachelor',
    'процент немосквичей': 'non_moscow_percentage',
    'средние доп баллы': 'avg_additional_points',
    'кол-во бронзы ГТО': 'num_bronze'
}

# Переименование столбцов
summary_df = summary_df.rename(columns=column_rename_map).drop('Unnamed: 0', axis=1)

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Столбцы переименованы. Общая таблица обновлена.")

## Объединеник столбцов

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Объединение столбцов "faculty" и "specialty"
summary_df['faculty_specialty'] = summary_df['faculty'] + ' - ' + summary_df['specialty']

# Помещение нового столбца на второе место
cols = list(summary_df.columns)  # Список текущих столбцов
cols.insert(1, cols.pop(cols.index('faculty_specialty')))  # Переместить новый столбец на второе место
summary_df = summary_df[cols]  # Перестроить DataFrame в нужном порядке

# Удаление столбцов "faculty" и "specialty"
summary_df.drop(['faculty', 'specialty'], axis=1, inplace=True)

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Обновленная общая таблица. Столбец 'faculty_specialty' на втором месте, 'faculty' и 'specialty' удалены.")

## Приведение данных к однородному виду (нет пропущенных годов у направлений)

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'  # Имя общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка общей таблицы
summary_df = pd.read_csv(summary_file_path)

# Определение уникальных специальностей за 2016, 2017, 2018 и 2019 годы
unique_specialties_2016 = set(summary_df[summary_df['year'] == 2016]['faculty_specialty'].unique())
unique_specialties_2017 = set(summary_df[summary_df['year'] == 2017]['faculty_specialty'].unique())
unique_specialties_2018 = set(summary_df[summary_df['year'] == 2018]['faculty_specialty'].unique())
unique_specialties_2019 = set(summary_df[summary_df['year'] == 2019]['faculty_specialty'].unique())

# Найти пересечение всех специальностей за 2016-2019 годы
specialties_to_keep = (
    unique_specialties_2016
    .intersection(unique_specialties_2017)
    .intersection(unique_specialties_2018)
    .intersection(unique_specialties_2019)
)

# Удаление всех строк, у которых специальность не в списке оставляемых
summary_df = summary_df[summary_df['faculty_specialty'].isin(specialties_to_keep)]

# Сохранение обновленной общей таблицы
summary_file_path = os.path.join(data_folder, summary_file_name)
summary_df.to_csv(summary_file_path, index=False)

print("Обновленная общая таблица содержит только специальности, присутствующие в 2016-2019.")

## Вывод датафрейма

In [ ]:
df = pd.read_csv('общая_таблица.csv')
df

# Разбиение на подвыборки

In [ ]:
import os
import pandas as pd

# Папка, где находятся исходные таблицы
input_folder = 'split_tables'  # Папка с исходными данными
output_folder_w = 'split_tables_w'  # Папка для таблиц с полом 0
output_folder_m = 'split_tables_m'  # Папка для таблиц с полом 1

# Убедитесь, что папки для сохранения существуют
os.makedirs(output_folder_w, exist_ok=True)
os.makedirs(output_folder_m, exist_ok=True)

# Получить список всех CSV-файлов в папке
input_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Разделение данных по значениям столбца "пол"
for input_file in input_files:
    file_path = os.path.join(input_folder, input_file)
    df = pd.read_csv(file_path)  # Загрузка данных из файла

    df['Пол'] = pd.to_numeric(df['Пол'], errors='coerce')
    # Разделить на женские (пол = 0) и мужские (пол = 1)
    female_df = df[df['Пол'] == 0]  # Строки, где пол = 0
    male_df = df[df['Пол'] == 1]  # Строки, где пол = 1

    # Создать уникальные имена файлов для сохранения
    base_name = os.path.splitext(input_file)[0]  # Имя файла без расширения

    # Сохранить женские данные в split_tables_w
    female_output_path = os.path.join(output_folder_w, base_name + '_w.csv')  # Добавить суффикс '_w'
    female_df.to_csv(female_output_path, index=False)  # Сохранить

    # Сохранить мужские данные в split_tables_m
    male_output_path = os.path.join(output_folder_m, base_name + '_m.csv')  # Добавить суффикс '_m'
    male_df.to_csv(male_output_path, index=False)  # Сохранить

print("Таблицы разделены по полу и сохранены в 'split_tables_m' и 'split_tables_w'.")

In [ ]:
import os
import pandas as pd

# Папка, где находятся исходные таблицы
input_folder = 'split_tables'  # Папка с исходными данными
output_folder_n = 'split_tables_n'  
output_folder_y = 'split_tables_y'  

# Убедитесь, что папки для сохранения существуют
os.makedirs(output_folder_n, exist_ok=True)
os.makedirs(output_folder_y, exist_ok=True)

# Получить список всех CSV-файлов в папке
input_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for input_file in input_files:
    file_path = os.path.join(input_folder, input_file)
    df = pd.read_csv(file_path)  # Загрузка данных из файла

    female_df = df[df['Право на общежитие'] == 'Нет']  
    male_df = df[df['Право на общежитие'] == 'Да']  

    # Создать уникальные имена файлов для сохранения
    base_name = os.path.splitext(input_file)[0]  # Имя файла без расширения

    # Сохранить женские данные в split_tables_n
    female_output_path = os.path.join(output_folder_n, base_name + '_n.csv')  # Добавить суффикс '_n'
    female_df.to_csv(female_output_path, index=False)  # Сохранить

    # Сохранить мужские данные в split_tables_y
    male_output_path = os.path.join(output_folder_y, base_name + '_y.csv')  # Добавить суффикс '_y'
    male_df.to_csv(male_output_path, index=False)  # Сохранить

print("Таблицы разделены по полу и сохранены в 'split_tables_n' и 'split_tables_y'.")

# Некоторые графики

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Пример загрузки вашего датафрейма
data = df

# Выбор только числовых столбцов
numeric_data = data.select_dtypes(include=[np.number])

# Гистограммы для числовых данных
numeric_data.hist(figsize=(10, 10), bins=20)
plt.tight_layout()
plt.show()

# Коробчатые диаграммы (Boxplots) для числовых данных
plt.figure(figsize=(10, 6))
sns.boxplot(data=numeric_data, orient='h')
plt.title("Коробчатые диаграммы для числовых столбцов")
plt.show()

# Матрица корреляции
plt.figure(figsize=(10, 8))
correlation_matrix = numeric_data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Матрица корреляции для числовых столбцов")
plt.show()

# Тепловая карта пропущенных данных
plt.figure(figsize=(10, 6))
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title("Пропущенные значения в датафрейме")
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица_y.csv'
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка данных
df = pd.read_csv(summary_file_path)

# Список столбцов для гистограмм
hist_columns = [
    'num_bvi', 'num_target_students', 'num_quota_students', 'num_student', 
    'passing_score', 'average_score', 'abbreviation_length', 'num_budget_places', 
    'gender', 'non_moscow_percentage', 'avg_additional_points'
]

# Создаем сетку подграфиков с достаточным пространством (4x3)
rows, cols = 4, 3  # Больше строк для более компактного размещения
fig, axes = plt.subplots(rows, cols, figsize=(18, 12))  # Увеличиваем figsize

# Индексируем подграфики, чтобы добавить гистограммы
for ax, column in zip(axes.flat, hist_columns):
    mean = df[column].mean()  # Среднее значение
    std = df[column].std()  # Стандартное отклонение

    # Определение границ по правилу трех сигм
    lower_limit = mean - 3 * std
    upper_limit = mean + 3 * std

    # Фильтрация данных по этим границам
    trimmed_df = df[(df[column] >= lower_limit) & (df[column] <= upper_limit)]

    # Построение гистограммы с KDE-графиком для обрезанных данных
    sns.histplot(trimmed_df[column], kde=True, bins=15, ax=ax)  # Построение гистограммы
    ax.set_title(f'Гистограмма для {column} (3 сигмы)', fontsize=12)  # Уменьшаем размер шрифта заголовка
    ax.set_xlabel(column, fontsize=10)  # Уменьшаем размер шрифта подписи оси
    ax.set_ylabel('Частота', fontsize=10)  # Уменьшаем размер шрифта подписи оси

# Подгоняем все графики, чтобы избежать наложения
plt.tight_layout()  
plt.show()  # Показываем все подграфики на одной картинке

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка данных
df = pd.read_csv(summary_file_path)

# Список столбцов с бинарными значениями для круговых диаграмм
binary_columns = ['faculty_humanities', 'specialist_or_bachelor']

# Создание подграфиков - один круговой график на строку
fig, axes = plt.subplots(len(binary_columns), 1, figsize=(10, 6))  # 1 график на строку

# Построение круговых диаграмм
for ax, column in zip(axes, binary_columns):
    # Получение распределения значений 0 и 1
    counts = df[column].value_counts()

    # Построение круговой диаграммы
    ax.pie(
        counts,
        labels=counts.index,  # Метки для сегментов
        autopct='%1.1f%%',  # Проценты с одним знаком после запятой
        startangle=90  # Начальный угол
    )
    
    ax.set_title(f'Соотношение значений в {column}')  # Заголовок для графика

# Обеспечение корректного размещения графиков
plt.tight_layout()

# Показ всех графиков на одной картинке
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка данных
df = pd.read_csv(summary_file_path)

# Выбор только числовых столбцов
numeric_df = df.select_dtypes(include=[np.number])  # Выбор только числовых данных

# Убедитесь, что все значения в числовых столбцах корректны
numeric_df = numeric_df.apply(pd.to_numeric, errors='coerce')  # Преобразование в числа, недопустимые значения заменяются на NaN

# Получение матрицы корреляции
correlation_matrix = numeric_df.corr()  # Вычисление корреляции

# Построение тепловой карты для матрицы корреляции
plt.figure(figsize=(12, 10))  # Установка размера фигуры
sns.heatmap(
    correlation_matrix, 
    annot=True,  # Добавление значений корреляции в каждую ячейку
    cmap='coolwarm',  # Цветовая шкала
    fmt=".2f",  # Форматирование (2 знака после запятой)
    linewidths=0.5,  # Толщина линий между ячейками
    linecolor='black',  # Цвет линий между ячейками
    cbar_kws={'label': 'Корреляция'}  # Метка для цветовой шкалы
)
plt.title('Матрица корреляции')  # Заголовок тепловой карты
plt.show()  # Показать график

In [ ]:
import pandas as pd

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка данных
df = pd.read_csv(summary_file_path)

# Считаем количество значений 0 в столбце 'num_budget_places'
count_zeros = (df['num_budget_places'] == 0).sum()

print(f"Количество значений 0 в столбце 'num_budget_places': {count_zeros}")

# Для теха

In [ ]:
import pandas as pd
import os

# Папка, где находится ваша общая таблица
data_folder = '.'  # Текущая папка
summary_file_name = 'общая_таблица.csv'
summary_file_path = os.path.join(data_folder, summary_file_name)

# Загрузка данных
df = pd.read_csv(summary_file_path)

# Получение описательных статистик для числовых столбцов
numeric_desc = df.describe().T  # Транспонированный вид для удобства

# Добавление столбца "Описание" с дополнительной информацией
numeric_desc['Описание'] = [
    'год', 'кол-во поступивших без вступительных экзаменов', 'кол-во поступивших по целевой квоте',
    'кол-во поступивших по особой квоте', 'кол-во участников конкурса', 'проходной балл второй волны',
    'среднее кол-во баллов по всем предметам', 'гуманитарность факультета',
    'длина названия факультета', 'кол-во бюджетных мест', 'пол (0 - жен., 1 - муж.)', 'специалитет или бакалавриат',
    'процент людей, имеющих право на общежитие', 'среднее кол-во дополнительных баллов', 'число результатов первокурсников на бронзу гто бег + лыжники'
]

# Переименование столбцов, чтобы они были как в примере
numeric_desc.rename(columns={
    'mean': 'Среднее',
    'std': 'Стандартное отклонение',
    'min': 'Минимум',
    'max': 'Максимум'
}, inplace=True)

# Форматирование в виде таблицы LaTeX
latex_table = numeric_desc.to_latex(index=False, float_format="%.2f")  # Экспорт в формат LaTeX

# Сохранение в файл LaTeX
latex_path = os.path.join(data_folder, 'описательные_статистики.tex')
with open(latex_path, 'w', encoding='utf-8') as f:
    f.write(latex_table)

print("Таблица описательных статистик сохранена в LaTeX.")